In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define the input image size and batch size
img_width, img_height = 224, 224
batch_size = 64

# Define the directory for the training data
train_data_dir = '/content/drive/MyDrive/AgeClassification/train/'

# Define the directories for the test data
test_data_dir = '/content/drive/MyDrive/AgeClassification/test/'

# Create an ImageDataGenerator for the training and validation sets with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,  # Use 20% of the training data for validation
)

# Create an ImageDataGenerator for the test set
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load the training and validation images from the directory
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=True,
    subset='training',  # Use the subset of the data for training
)

val_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
    subset='validation',  # Use the subset of the data for validation
)

# Load the test images from the directory
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    classes=['children', 'adults'],
    shuffle=False,
)

# Load the pre-trained ResNet50 model and remove the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add a global average pooling layer and a dense output layer with sigmoid activation
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

# Define the full model by combining the base model and the added layers
model = Model(inputs=base_model.input, outputs=x)

# Freeze the layers of the pre-trained ResNet50 model to prevent retraining
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data and validate on the validation data
model.fit(train_generator, epochs=100,
          validation_data=val_generator,
          callbacks= [tf.keras.callbacks.EarlyStopping(
              patience=5, #
              monitor='val_accuracy',
              restore_best_weights=True
          )])

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, verbose=2)

# Print the test accuracy
print('Test accuracy:', test_acc)

Found 553 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
Epoch 1/100
9/9 [==============================] - 130s 14s/step - loss: 0.7256 - accuracy: 0.5588 - val_loss: 0.6610 - val_accuracy: 0.6159
Epoch 2/100
9/9 [==============================] - 99s 11s/step - loss: 0.5399 - accuracy: 0.7197 - val_loss: 0.5544 - val_accuracy: 0.7101
Epoch 3/100
9/9 [==============================] - 105s 12s/step - loss: 0.4784 - accuracy: 0.7776 - val_loss: 0.5010 - val_accuracy: 0.7464
Epoch 4/100
9/9 [==============================] - 121s 14s/step - loss: 0.4165 - accuracy: 0.8174 - val_loss: 0.4572 - val_accuracy: 0.7899
Epoch 5/100
9/9 [==============================] - 100s 12s/step - loss: 0.3939 - accuracy: 0.8300 - val_loss: 0.4483 - val_accuracy: 0.8333
Epoch 6/100
9/9 [==============================] - 101s 11s/step - loss: 0.3779 - accuracy: 0.8391 - val_loss: 0.4362 - val_accuracy: 0.7899
Epoch 7/100
9/9 [===============

#Save Model

In [ ]:
model.save('/content/drive/MyDrive/AgeClassification/modelAgeClassificationV3_Full.h5')
